<a href="https://colab.research.google.com/github/mars241/NLP_Natural-Language-Processing/blob/main/AAM_WCS_NLP_4_Bag_of_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🦾 Challenge - Tweetons !
Entraîne-toi à classifier des tweets pour dire s'ils sont positifs ou négatifs. Ca pourrait être un outil très utile pour optimiser le travail d'un community manager.

1. Importe 
[cet ensemble de données de tweets](https://github.com/murpi/wilddata/raw/master/quests/tweets.zip) dans un DataFrame.

2. Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?
3. Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32 et un train_size de 0.75.
4. Crée un modèle vectorizer avec scikit-learn en utilisant la méthode Countvectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.
5. Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.966 pour le test d'entraînement, et 0.877 pour l'ensemble de test.
6. Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

# Critères de validation
Le notebook est consultable pour toute personne disposant du lien fourni par l'élève.
CountVectorizer est utilisée pour le preprocessing
La classification de tweets est faite grâce à un algorithme de régression logistique.
La performance de l'algorithme de régression logistique est mesurée.
Toutes les étapes sont réalisées.


In [2]:
# importation des bibliothèques pour la lecture & traitement du DataFram
import pandas as pd
import numpy as np

In [3]:
# Librarie / Module importation fichier
from google.colab import files

# Création de la variable mommé ici "uploaded", qui contiendra ici votre fichier DF ("train.csv")
uploaded = files.upload()

Saving train.csv to train.csv


1. Importe cet ensemble de données de tweets dans un DataFrame

In [30]:
# Lire et afficher le datafram
import pandas as pd
df = pd.read_csv("train.csv")
df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [25]:
# plus d'info sur le contenu du DF
df.info()

# 4 colonnes & 27480 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [31]:
# la colonne "sentiment" semble nous intéressé, vérifions les valeurs possibles qu'elle contient:
df['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

2. Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?

In [32]:
# Remplace dans la colonne "sentiment" les "neutral"(s) par des nan
df = df.replace({'sentiment': ['neutral']}, np.nan)

# Supprimer les lignes qui contiennent np.nan
df = df.dropna()
     

In [33]:
# Datafram des tweets négatifs
df_negative = df['sentiment'] == 'negative'
negative_tweets = df.loc[df_negative]
negative_tweets

,textID,text,selected_text,sentiment
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative
...,...,...,...,...
27463,e7eecffdc8,rec game....trying not to cry...the pain is to...,breakingg,negative
27469,778184dff1,lol i know and haha..did you fall asleep?? o...,t bored,negative
27471,8f5adc47ec,http://twitpic.com/663vr - Wanted to visit the...,were too late,negative
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative


In [34]:
# Datafram des tweets positifs
df_positive = df['sentiment'] == 'positive'
positive_tweets = df.loc[df_positive]
positive_tweets

,textID,text,selected_text,sentiment
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive
11,16fab9f95b,I really really like the song Love Story by Ta...,like,positive
21,e48b0b8a23,Playing Ghost Online is really interesting. Th...,interesting.,positive
25,e00c6ef376,"the free fillin` app on my ipod is fun, im add...","the free fillin` app on my ipod is fun, im add...",positive
...,...,...,...,...
27466,432e6de6c9,morning twit-friends! welcome to my new followers,welcome,positive
27473,8f14bb2715,So I get up early and I feel good about the da...,I feel good ab,positive
27474,b78ec00df5,enjoy ur night,enjoy,positive
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive


2. (suite) Quel est le pourcentage de tweets positifs/négatifs ?

In [35]:
# Calcule du pourcentage de tweets positifs
percent_positive = (len(positive_tweets) / len(df))*100

# Calcule du pourcentage de tweets negatifs 
percent_negative = (len(negative_tweets) / len(df))*100

print("Le pourcentage de tweets positifs :",percent_positive)
print("Le pourcentage de tweets negatifs :",percent_negative)

# RMQ : On a plus de tweets positifs que négatifs

Le pourcentage de tweets positifs : 52.44759518425717
Le pourcentage de tweets negatifs : 47.55240481574283


In [36]:
# Autrement et mieux (on fait ressembler la valeur à un taux grâce à la fonction de format "f" & | "lamba"):
# NB: le cacule se fait avec le DF originel dépouillé des nan !

# import numpy as np

# Calcule du pourcentage de tweets positifs
percent_positive_per = df[df['sentiment'] != 'negative']['sentiment'].count() / len(df)*100
print(f'Il y a {round(percent_positive_per,1)} pourcentage de tweets positifs')
#percent_positive_per = (len(positive_tweets) / len(df)).apply(lambda x: f"{round(x * 100)} % ")

# Calcule du pourcentage de tweets negatifs 
percent_negative_per = df[df['sentiment'] != 'positive']['sentiment'].count() / len(df)*100
print(f'Il y a {round(percent_negative_per,1)} pourcentage de tweets négatifs')
#percent_negative_per = (len(negative_tweets) / len(df)).apply(lambda x: f"{round(x * 100)} % ")

Il y a 52.4 pourcentage de tweets positifs
Il y a 47.6 pourcentage de tweets négatifs


3. Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32 et un train_size de 0.75.

In [18]:
# importation de la bibliothèque du ML pour le " train test split"
from sklearn.model_selection import train_test_split

X = df['text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=32, train_size = 0.75)

4. Crée un modèle vectorizer avec scikit-learn en utilisant la méthode Countvectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.

In [19]:
# Créer un modèle vectoriel avec scikit-learn en utilisant la méthode Countvectorizer.
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# Entraîner le modèle sur X_train
X_train_CV = vectorizer.fit_transform(X_train)

# Créer la matrice X_test_CV sans ré-entraînement du modèle
X_test_CV = vectorizer.transform(X_test)

X_train_CV

<12272x15806 sparse matrix of type '<class 'numpy.int64'>'
	with 144578 stored elements in Compressed Sparse Row format>

5. Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.966 pour le test d'entraînement, et 0.877 pour l'ensemble de test.

In [21]:
# Entraîner une régression logistique
from sklearn.linear_model import LogisticRegression

# Ici, nous entraînons le modèle uniquement sur l'ensemble de données d'entraînement.
model = LogisticRegression()

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train_CV, y_train)


# Et maintenant nous comparons les deux scores :
print("\nLe Score pour l'ensemble de données Train :", model.score(X_train_CV, y_train))
print("Le Score pour l'ensemble de données Test :", model.score(X_test_CV, y_test))


Le Score pour l'ensemble de données Train : 0.9663461538461539
Le Score pour l'ensemble de données Test : 0.8772916157418724


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:450: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  opt_res = optimize.minimize(


6. Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

In [22]:
# Afficher 10 tweets mals prédits (faux positifs ou faux négatifs).

predictions = model.predict(X_test_CV)

# Obtenir les indices des tweets incorrectement prédits
y_testArray = y_test.to_numpy()
X_testArray = X_test.to_numpy()

incorrect_indices = [i for i in range(len(y_test)) if y_testArray[i] != predictions[i]]

# Imprimer les tweets qui ont été mal prédits
for i in incorrect_indices[:10]:
  print(f'Tweet: {X_testArray[i]}')
  print(f'Prediction: {predictions[i]}')
  print(f'Actual: {y_testArray[i]}')
  print()

Tweet: HollowbabesHere comes the utter shite #bgt <I completely agree
Prediction: positive
Actual: negative

Tweet:  SUFFICATION NO BREATHING. It`s okay. There`ll be more. You`re invited to mine, but I can`t promise fun times.  *Jinx
Prediction: positive
Actual: negative

Tweet: i wanna vote for Miley Cyrus for the mtv movie awards..but i don`t know where i could  somebody could send me a link? thaank you <3
Prediction: negative
Actual: positive

Tweet: I love music so much that i`ve gone through pain to play :S my sides of my fingers now are peeling and have blisters from playing so much
Prediction: positive
Actual: negative

Tweet: I can only message those who message me, if we`re fwends...so those that want replies..follow me.  hmm..that sounds funny..
Prediction: negative
Actual: positive

Tweet: wish I could feel no pain (8)  but it`s ok, at least they like Brazil!
Prediction: negative
Actual: positive

Tweet:  so glad i`m not at uni anymore
Prediction: negative
Actual: positive

